In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, when, regexp_replace
from pyspark.sql.types import IntegerType, DoubleType

# --- 1. Configuração do Spark e Variáveis ---
spark = SparkSession.builder \
    .appName("SilverLayerProcessingOptimized") \
    .getOrCreate()

BRONZE_SUICIDE_PATH = 'hdfs://namenode:9870/datalake/bronze/raw_suicidio.json'
BRONZE_DEPRESSION_PATH = 'hdfs://namenode:9870/datalake/bronze/raw_depressao.json'
SILVER_PATH = 'hdfs://namenode:9870/datalake/silver/who_data_silver.parquet'

INDICATORS_MAP = {
    'SDGSUICIDE': 'Taxa_Mortalidade_Suicidio',
    'MH_12': 'Prevalencia_Transtorno_Humor'
}

# Este é o filtro chave para remover as subdivisões de género/sexo
AGGREGATE_SEX_CODE = 'BOTHSEXES' 

def load_bronze_data(path):
    """Carrega o JSON bruto e achata a estrutura aninhada."""
    print(f"Lendo dados brutos de: {path}")
    df_raw = spark.read.json(path, multiLine=True)
    df_exploded = df_raw.selectExpr("explode(value) as data")
    
    # Seleciona os campos aninhados que nos interessam, incluindo a dimensão de Sexo (Dim1) se existir
    df_selected = df_exploded.select(
        col("data.GHO").alias("GHO_ID"),
        col("data.YEAR").alias("Year"),
        col("data.COUNTRY").alias("Country_Code"),
        col("data.NUMERIC_VALUE").alias("Value"),
        col("data.Dim1").alias("Sex_Dimension") # Incluímos para filtrar imediatamente
    )
    return df_selected

def transform_to_silver(df_suicide, df_depression):
    """Aplica transformações de limpeza e filtra a população total (BOTHSEXES)."""
    print("Iniciando limpeza, unificação e filtragem (Camada Silver Otimizada)...")
    
    # 1. União dos dados (Suicídio e Depressão)
    df_combined = df_suicide.unionByName(df_depression)

    # 2. FILTRO CHAVE: Manter apenas os dados agregados (Ambos os Sexos)
    df_filtered = df_combined.filter(col("Sex_Dimension") == AGGREGATE_SEX_CODE)

    # 3. Mapeamento de Indicadores
    df_silver = df_filtered.withColumn(
        "Indicator_Name",
        when(col("GHO_ID") == "SDGSUICIDE", INDICATORS_MAP['SDGSUICIDE'])
        .when(col("GHO_ID") == "MH_12", INDICATORS_MAP['MH_12'])
        .otherwise(col("GHO_ID"))
    )

    # 4. Tratamento de Tipos de Dados e Nulos
    df_silver = df_silver.withColumn("Year", col("Year").cast(IntegerType())) \
                         .withColumn("Value", col("Value").cast(DoubleType()))
    df_silver = df_silver.dropna(subset=['Value', 'Year'])
    
    # 5. Seleção final das colunas (eliminando a coluna de Sexo)
    df_final = df_silver.select(
        "Country_Code", 
        "Year", 
        "Indicator_Name", 
        "Value"
    )
    
    print(f"Limpeza e Filtragem de Sexo concluída. Total de linhas na Silver: {df_final.count()}")
    return df_final

def save_silver_data(df_silver, file_path):
    """Salva o DataFrame limpo no formato Parquet no HDFS."""
    if df_silver.count() > 0:
        print(f"Salvando dados limpos (Camada Silver) em: {file_path}")
        df_silver.write.mode("overwrite").parquet(file_path)
        print("Dados salvos com sucesso!")


if __name__ == "__main__":
    df_suicide = load_bronze_data(BRONZE_SUICIDE_PATH)
    df_depression = load_bronze_data(BRONZE_DEPRESSION_PATH)
    
    if df_suicide.count() > 0 and df_depression.count() > 0:
        df_silver = transform_to_silver(df_suicide, df_depression)
        save_silver_data(df_silver, SILVER_PATH)
    else:
        print("Não foi possível processar. Verifique se os arquivos Bronze existem no HDFS.")
    
    spark.stop()

'pip' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.


ModuleNotFoundError: No module named 'pyspark'

: 